In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('AMES_TRAIN.csv')
test = pd.read_csv('AMES_TEST_SFAM.csv')
train.columns = [s.lower() for s in train.columns]
test.columns = [s.lower() for s in test.columns]
train['qualityindex'] = (train.overallqual*train.overallcond)
train['totalsqftcalc'] = (train.bsmtfinsf1+train.bsmtfinsf2+train.grlivarea)
test['qualityindex'] = (test.overallqual*test.overallcond)
test['totalsqftcalc'] = (test.bsmtfinsf1+test.bsmtfinsf2+test.grlivarea)
train=train.replace({'totalsqftcalc': {np.NaN : 10000}})
train=train.replace({'lotarea': {np.NaN : 10000}})
train=train.replace({'lotfrontage': {np.NaN : 10000}})
test=test.replace({'totalsqftcalc': {np.NaN : 10000}})
test=test.replace({'lotarea': {np.NaN : 10000}})
test=test.replace({'lotfrontage': {np.NaN : 10000}})

In [ ]:
train = train[train['totalsqftcalc'] < 5000]
train = train[train['salecondition'] == 'Normal']

In [ ]:
#train = train[train['roofstyle'] != 'Gable']
#train = train[train['roofstyle'] != 'Hip']

In [ ]:
#print(train.head())
#print(test.head())

In [ ]:
X = train[['saleprice','qualityindex','totalsqftcalc','yearbuilt','lotarea','lotfrontage']].copy()
X1 = train[['qualityindex','totalsqftcalc','yearbuilt','lotarea','lotfrontage']].copy()
corr = X[X.columns].corr()
#corr

In [ ]:
Y = train[['saleprice']].copy()
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
select_top_3 = SelectKBest(score_func=chi2, k = 3)
fit = select_top_3.fit(X1,Y)
features = fit.transform(X1)
#features[0:5] 

In [ ]:
train.plot.scatter(y='saleprice', x='totalsqftcalc') 
#This lets me see some outliars that could be distorting the data.

In [ ]:
train.plot.scatter(y='totalsqftcalc', x='saleprice') 

In [ ]:
train.plot.scatter(y='saleprice', x='totalsqftcalc') 

In [274]:
model1 = smf.ols(formula='Y ~ qualityindex+totalsqftcalc+C(lotconfig)+C(housestyle)+yearbuilt', data=train).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     487.1
Date:                Sun, 10 Feb 2019   Prob (F-statistic):               0.00
Time:                        21:33:33   Log-Likelihood:                -19686.
No. Observations:                1677   AIC:                         3.940e+04
Df Residuals:                    1662   BIC:                         3.948e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept               -1.476e+06   5.88e+04    -25.124      0.000   -1.59e+06   -1.36e+06
C(lotconfig)[T.CulDSac] -1806.7515   3648.330     -0.495      0.621   -8962.558    5349.055
C(lotconfig)[T.FR2]     -1.386e+04   4800.658     -2.888      0.004   -2.33e+04   -4448.964
C(lotconfig)[T.FR3]     -1.152e+04    1.1e+04     -1.050      0.294   -3.31e+04       1e+04
C(lotconfig)[T.Inside]   -361.8839   2026.118     -0.179      0.858   -4335.896    3612.128
C(housestyle)[T.1.5Unf]  1.356e+04   8494.219      1.596      0.111   -3099.560    3.02e+04
C(housestyle)[T.1Story] -1763.0883   2662.794     -0.662      0.508   -6985.872    3459.695
C(housestyle)[T.2.5Fin]  2.325e+04   1.55e+04      1.499      0.134   -7173.318    5.37e+04
C(housestyle)[T.2.5Unf]  9775.0872   8480.450      1.153      0.249   -6858.403    2.64e+04
C(housestyle)[T.2Story] -5906.3610   2856.310     -2.068      0.039   -1.15e+04    -304.017
C(housestyle)[T.SFoyer] -2.663e+04   4997.404     -5.329      0.000   -3.64e+04   -1.68e+04
C(housestyle)[T.SLvl]   -1.743e+04   4294.848     -4.058      0.000   -2.59e+04   -9004.895
qualityindex             2284.5286     89.399     25.554      0.000    2109.181    2459.876
totalsqftcalc              58.6757      1.268     46.286      0.000      56.189      61.162
yearbuilt                 743.1046     30.764     24.155      0.000     682.765     803.444
==============================================================================
Omnibus:                      369.813   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1353.342
Skew:                           1.045   Prob(JB):                    1.34e-294
Kurtosis:                       6.873   Cond. No.                     2.23e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.23e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#previous way I was doing VIF
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
#VIF for Model 1
# Break into left and right hand side; y and X
Y, X = dmatrices( "Y ~ qualityindex+totalsqftcalc+C(lotconfig)+C(housestyle)+yearbuilt", data=train, return_type="dataframe")
# For each Xi, calculate VIF
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# Fit X to y
result = smf.OLS(Y, X).fit()
#print(vif)

In [ ]:
#Model 1 VIF (this way I was able to label each output)
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif = pd.Series(vif, index =X.columns)
print("This is the VIF for Model 1")
print(vif)
#dropped Hip and Gable from roofstyles due to very high vifs at 39 and 37 respectively
#dropped GasA and GasW from heating due to high vifs as well at 28 and 21 respectively

In [ ]:
model2 = smf.ols(formula='Y ~ qualityindex+totalsqftcalc+yearbuilt', data=train).fit()
model2.summary()

In [ ]:
#VIF for Model 2
# Break into left and right hand side; y and X
Y, X = dmatrices( "Y ~ qualityindex+totalsqftcalc+yearbuilt ", data=train, return_type="dataframe")
# For each Xi, calculate VIF
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# Fit X to y
result = smf.OLS(Y, X).fit()
#print(vif)
#Constant is 4574.1026694
#VIF for qualityindex is 1.152
#VIF for totalsqftcalc is 1.238
#VIF for yearbuilt is 1.125

In [ ]:
#Model 2 VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif = pd.Series(vif, index =X.columns)
print("This is the VIF for Model 2")
print(vif)

In [ ]:
predictions_1 = model1.fittedvalues
predictions_1.head()

In [ ]:
predictions_2 = model2.fittedvalues
predictions_2.head()

In [ ]:
test_predictions_1 = model1.predict(test)
d = {'p_saleprice': test_predictions_1}
df1 = test[['index']]
df2=pd.DataFrame(data=d)
Strouse_TestPredictions_1 = pd.concat([df1,df2],axis = 1, join_axes=[df1.index])
Strouse_TestPredictions_1.head()

In [ ]:
Strouse_TestPredictions_1.to_csv('logan_strouse_HW2_1scored.csv')

In [ ]:
#dir(model2)

In [ ]:
y = np.array(train['saleprice'])
X = train[['qualityindex','totalsqftcalc','yearbuilt','yearremodel','wooddecksf','openporchsf' ]].copy()
print(X.head())
#model = feature_selection.SelectKBest(score_func.feature_selection.f_regression, k=3) #you will need to convert y to an array
#results = model.fit(X,y)
#scores are the F values for each variable, bigger is better
#results.scores

In [ ]:

#the next code is using my model2 in the shell code

train_predictions = model2.fittedvalues
train_predictions.head()
tr = {'p_saleprice': train_predictions}
df1 = train[['index','saleprice','neighborhood','totalsqftcalc']]
df2=pd.DataFrame(data=tr)
n_file = pd.concat([df1,df2],axis = 1, join_axes=[df1.index])

n_file['residual'] = (n_file.saleprice-n_file.p_saleprice)
n_file['a_ppsf'] = (n_file.saleprice/n_file.totalsqftcalc)
n_file['p_ppsf'] = (n_file.p_saleprice/n_file.totalsqftcalc)
n_file
import seaborn as sns
ax = sns.boxplot(x="neighborhood", y="residual", data=n_file)
ax.set_title("Boxplot of Resdiuals by Neighborhood")
ax.set_ylabel("Residual")
plt.show()

g = n_file.groupby('neighborhood').mean()
print(g)

#To set up the indicator variables try this for your neighborhood groups...

#group1 = ['SWISU', 'IDOTRR', 'OldTown' ]

#n = n_file.copy()

#n['group1'] = n['neighborhood'].isin(['SWISU', 'IDOTRR', 'OldTown' ]).astype('int64')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size = 0.3)